# Nobel Laureates 1901 - 2016
Zestawienie danych pobranych z https://www.kaggle.com/datasets/nobelfoundation/nobel-laureates?select=archive.csv

Analizie poddano zależność ilości kobiet wśród laureatów nagrody Nobla. W szczególności analizowan0:
1. Zależność od kategorii, w której otrzymywana była nagroda od płci
2. Zależność od kategorii jak i roku otrzymania nagrody od płci

Na koniec podsumowano zestawienie i uwzględniono ogólno dostępne fakty wyjaśniające pradopodobne przyczyny takiego stanu.

In [1]:
# import package to analysis
import numpy as np
import pandas as pd
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# init_notebook_mode - to show plotly in Notebook's output
init_notebook_mode(connected = True)
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import subplots
import plotly.figure_factory as ff

In [2]:
# import nobel laureates dataset
laureates = pd.read_csv('archive.csv', sep = ',')
print("First rows in Nobel laureates dataset")
display(laureates.head())
print("Description of Nobel laureates dataset")
display(laureates.describe())
print("Info of Nobel laureates dataset")
display(laureates.info())

First rows in Nobel laureates dataset


,Year,Category,Prize,Motivation,Prize Share,Laureate ID,Laureate Type,Full Name,Birth Date,Birth City,Birth Country,Sex,Organization Name,Organization City,Organization Country,Death Date,Death City,Death Country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France


Description of Nobel laureates dataset


,Year,Laureate ID
count,969.000000,969.000000
mean,1970.287926,470.152735
std,32.937498,274.586623
min,1901.000000,1.000000
25%,1947.000000,230.000000
50%,1976.000000,462.000000
75%,1999.000000,718.000000
max,2016.000000,937.000000


Info of Nobel laureates dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 969 entries, 0 to 968
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Year                  969 non-null    int64 
 1   Category              969 non-null    object
 2   Prize                 969 non-null    object
 3   Motivation            881 non-null    object
 4   Prize Share           969 non-null    object
 5   Laureate ID           969 non-null    int64 
 6   Laureate Type         969 non-null    object
 7   Full Name             969 non-null    object
 8   Birth Date            940 non-null    object
 9   Birth City            941 non-null    object
 10  Birth Country         943 non-null    object
 11  Sex                   943 non-null    object
 12  Organization Name     722 non-null    object
 13  Organization City     716 non-null    object
 14  Organization Country  716 non-null    object
 15  Death Da

None

## 1. Podział na dyscypliny/kategorie nagrody z uwzględnieniem płci

In [3]:
category_and_sex = laureates[['Sex', 'Category', 'Full Name', 'Year']]
display(category_and_sex.head())

,Sex,Category,Full Name,Year
0,Male,Chemistry,Jacobus Henricus van 't Hoff,1901
1,Male,Literature,Sully Prudhomme,1901
2,Male,Medicine,Emil Adolf von Behring,1901
3,Male,Peace,Jean Henry Dunant,1901
4,Male,Peace,Frédéric Passy,1901


In [4]:
# analysis
laureates_by_category_and_sex = category_and_sex.pivot_table(index = {'Category', 'Sex'}, aggfunc = 'count')['Year'].to_frame()
laureates_by_category_and_sex.rename(columns = {'Year' : 'number_of_prize'}, inplace = True)
laureates_by_category_and_sex.reset_index(inplace = True)
display(laureates_by_category_and_sex)

,Category,Sex,number_of_prize
0,Chemistry,Female,4
1,Chemistry,Male,190
2,Economics,Female,2
3,Economics,Male,81
4,Literature,Female,14
5,Literature,Male,99
6,Medicine,Female,12
7,Medicine,Male,215
8,Peace,Female,16
9,Peace,Male,88


In [5]:
female_laureates_by_category = laureates_by_category_and_sex[laureates_by_category_and_sex['Sex'] == 'Female']
male_laureates_by_category = laureates_by_category_and_sex[laureates_by_category_and_sex['Sex'] == 'Male']
display(female_laureates_by_category)
display(male_laureates_by_category)

,Category,Sex,number_of_prize
0,Chemistry,Female,4
2,Economics,Female,2
4,Literature,Female,14
6,Medicine,Female,12
8,Peace,Female,16
10,Physics,Female,2


,Category,Sex,number_of_prize
1,Chemistry,Male,190
3,Economics,Male,81
5,Literature,Male,99
7,Medicine,Male,215
9,Peace,Male,88
11,Physics,Male,220


In [6]:
laureates_all_categories_by_sex = female_laureates_by_category.merge(male_laureates_by_category, how = 'outer', on = 'Category')
laureates_all_categories_by_sex.rename(columns = {'number_of_prize_x' : 'female', 'number_of_prize_y' : 'male'}, inplace = True)
laureates_all_categories_by_sex.drop(columns = {'Sex_x', 'Sex_y'}, inplace = True)
display(laureates_all_categories_by_sex)

,Category,female,male
0,Chemistry,4,190
1,Economics,2,81
2,Literature,14,99
3,Medicine,12,215
4,Peace,16,88
5,Physics,2,220


In [7]:
# display as plots
domain = laureates_all_categories_by_sex['Category']
fig = go.Figure()
fig.add_trace(go.Bar(x = domain, y = laureates_all_categories_by_sex['female'], name='female'))
fig.add_trace(go.Bar(x = domain, y = laureates_all_categories_by_sex['male'], name = 'male'))
fig.update_layout(title = 'Zestawienie danych 1901-2016 otrzymania nagrody Nobla dla różnych kategorii z uwzględnieniem płci')
iplot(fig)

In [8]:
# add columns all_laureates oraz percent_of_female/male
def all_laureates(row):
    row['all_laureates'] = row['female'] + row['male']
    return row

def percent_of_sex(row, sex):
    row[f'percent_of_{sex}'] = (row[sex] * 100) / row['all_laureates']
    return row

laureates_all_categories_by_sex = laureates_all_categories_by_sex.apply(lambda x: all_laureates(x), axis = 1)
display(laureates_all_categories_by_sex)
laureates_all_categories_by_sex = laureates_all_categories_by_sex.apply(lambda x: percent_of_sex(x, 'female'), axis = 1)
laureates_all_categories_by_sex = laureates_all_categories_by_sex.apply(lambda x: percent_of_sex(x, 'male'), axis = 1)
display(laureates_all_categories_by_sex)

,Category,female,male,all_laureates
0,Chemistry,4,190,194
1,Economics,2,81,83
2,Literature,14,99,113
3,Medicine,12,215,227
4,Peace,16,88,104
5,Physics,2,220,222


,Category,female,male,all_laureates,percent_of_female,percent_of_male
0,Chemistry,4,190,194,2.061856,97.938144
1,Economics,2,81,83,2.409639,97.590361
2,Literature,14,99,113,12.389381,87.610619
3,Medicine,12,215,227,5.286344,94.713656
4,Peace,16,88,104,15.384615,84.615385
5,Physics,2,220,222,0.900901,99.099099


In [9]:
# display as plots
domain = laureates_all_categories_by_sex['Category']
fig = go.Figure()
fig.add_trace(go.Bar(x = domain, y = laureates_all_categories_by_sex['percent_of_female'], name='female'))
fig.add_trace(go.Bar(x = domain, y = laureates_all_categories_by_sex['percent_of_male'], name = 'male'))
fig.update_layout(title = 'Zestawienie danych 1901-2016 otrzymania nagrody Nobla dla różnych kategorii z uwzględnieniem płci')
iplot(fig)

In [10]:
female_physics = laureates[((laureates['Sex'] == 'Female')  & (laureates['Category'] == 'Physics'))]
female_physics

,Year,Category,Prize,Motivation,Prize Share,Laureate ID,Laureate Type,Full Name,Birth Date,Birth City,Birth Country,Sex,Organization Name,Organization City,Organization Country,Death Date,Death City,Death Country
19,1903,Physics,The Nobel Prize in Physics 1903,"""in recognition of the extraordinary services ...",1/4,6,Individual,"Marie Curie, née Sklodowska",1867-11-07,Warsaw,Russian Empire (Poland),Female,NaN,NaN,NaN,1934-07-04,Sallanches,France
368,1963,Physics,The Nobel Prize in Physics 1963,"""for their discoveries concerning nuclear shel...",1/4,79,Individual,Maria Goeppert Mayer,1906-06-28,Kattowitz (Katowice),Germany (Poland),Female,University of California,"San Diego, CA",United States of America,1972-02-20,"San Diego, CA",United States of America


## Analiza ze względu na kategorię i płeć
Z powyższych zestawień jasno wynika, że mężczyźni dominują znacząco jako laureacie każdej kategorii nagrody Nobla. Największa przewaga jest widoczna w fizyce, gdzie tylko dwie kobiety otrzymały nagrodę Nobla: w 1903 roku Maria Sklodowska-Curie oraz w 1963 Maria Goeppert Mayer. Pierwsza była polką urodzoną w Warszawie, wówczas zaborze Rosyjskim, druga niemką, co ciekawe również urodzoną na terenie obecnej Polski, w Katowicach, wówczas był to zabór Niemiecki. Pani Maria Goeppert Mayer swoje badania prowadziła w USA, natomiast Maria Sklodowska-Curie we Francji (co nie zostalo zaznaczone w tym zestawieniu). Zestawienie to nie uwzględnia również dwóch innych laureatek nagrody Nobla w dziedzinie fizyki, kanadyjki Donna Strickland, która otrzymała go w 2018 oraz Andrea Ghez, która otrzymała go dwa lata później, bo w 2020.
Tylko w dwóch kategoriach: Literatura oraz Pokojowa Nagorda Nobla kobiety stanowią ponad 10% wszystkich nagrodzonych w tych kategoriach. W medycynie stanowią niewiele ponad 5% wszystkich laureatów. W pozostałych kategoriach stanowi to ok. 2%, poza fizyką, gdzie kobiety nie przekroczyły nawet 1%.
Aby mieć szerszy pogląd na to dlaczego jest zauważalna tak duża dysproporcja przeanalizowany zostanie jeszcze roku uzyskania nagrody dla każdej z płci.

## 2. Podział na dyscypliny/kategorie nagrody z uwzględnieniem płci i roku otrzymania nagrody

In [11]:
# analysis
laureates_by_category_sex_year = category_and_sex.pivot_table(index = {'Category', 'Year', 'Sex'}, aggfunc = 'count')
laureates_by_category_sex_year.rename(columns = {'Full Name' : 'number_of_prize'}, inplace = True)
laureates_by_category_sex_year.reset_index(inplace = True)
display(laureates_by_category_sex_year)

,Category,Sex,Year,number_of_prize
0,Chemistry,Female,1911,1
1,Chemistry,Female,1935,1
2,Chemistry,Female,1964,1
3,Chemistry,Female,2009,1
4,Chemistry,Male,1901,1
...,...,...,...,...
576,Physics,Male,2012,4
577,Physics,Male,2013,2
578,Physics,Male,2014,4
579,Physics,Male,2015,2


In [12]:
female = laureates_by_category_sex_year[laureates_by_category_sex_year['Sex'] == 'Female']
male = laureates_by_category_sex_year[laureates_by_category_sex_year['Sex'] == 'Male']
display(female.head())
display(male.head())

,Category,Sex,Year,number_of_prize
0,Chemistry,Female,1911,1
1,Chemistry,Female,1935,1
2,Chemistry,Female,1964,1
3,Chemistry,Female,2009,1
110,Economics,Female,2009,2


,Category,Sex,Year,number_of_prize
4,Chemistry,Male,1901,1
5,Chemistry,Male,1902,1
6,Chemistry,Male,1903,1
7,Chemistry,Male,1904,1
8,Chemistry,Male,1905,1


In [13]:
nobel = female.merge(male, how = 'outer', on = ['Category', 'Year'])
nobel.rename(columns = {'number_of_prize_x' : 'female', 'number_of_prize_y' : 'male'}, inplace = True)
nobel.drop(columns = {'Sex_x', 'Sex_y'}, inplace = True)
nobel.fillna(0.0, inplace = True)
nobel_sort_by_year = nobel.sort_values(by = ['Year'])
display(nobel_sort_by_year.head())

,Category,Year,female,male
196,Literature,1901,0.0,1.0
291,Medicine,1901,0.0,1.0
453,Physics,1901,0.0,1.0
45,Chemistry,1901,0.0,1.0
387,Peace,1901,0.0,2.0


In [14]:
nobel_laureates = nobel_sort_by_year.apply(lambda x: all_laureates(x), axis = 1)
display(nobel_laureates.head())
nobel_laureates = nobel_laureates.apply(lambda x: percent_of_sex(x, 'female'), axis = 1)
nobel_laureates = nobel_laureates.apply(lambda x: percent_of_sex(x, 'male'), axis = 1)
display(nobel_laureates.head())

,Category,Year,female,male,all_laureates
196,Literature,1901,0.0,1.0,1.0
291,Medicine,1901,0.0,1.0,1.0
453,Physics,1901,0.0,1.0,1.0
45,Chemistry,1901,0.0,1.0,1.0
387,Peace,1901,0.0,2.0,2.0


,Category,Year,female,male,all_laureates,percent_of_female,percent_of_male
196,Literature,1901,0.0,1.0,1.0,0.0,100.0
291,Medicine,1901,0.0,1.0,1.0,0.0,100.0
453,Physics,1901,0.0,1.0,1.0,0.0,100.0
45,Chemistry,1901,0.0,1.0,1.0,0.0,100.0
387,Peace,1901,0.0,2.0,2.0,0.0,100.0


In [15]:
# prepared plot
x = nobel_laureates['Year']
y = nobel_laureates['Category']
z_female = nobel_laureates['female']
z_male = nobel_laureates['male']

#female map
fig = go.Figure()
fig.add_trace(go.Heatmap(x = x,
                         y = y,
                         z = z_female))
fig.update_layout(title = 'Zestawienie ilości kobiet, które otrzymały nagrodę Nobla w zależności od roku oraz kategorii')
iplot(fig)

#male map
fig = go.Figure()
fig.add_trace(go.Heatmap(x = x,
                         y = y,
                         z = z_male))
fig.update_layout(title = 'Zestawienie ilości mężczyzn, którzy otrzymali nagrodę Nobla w zależności od roku oraz kategorii')
iplot(fig)

### UWAGA!
Puste pola oznaczają, że w danym roku nie było laureata nagrody w danej kategorii

In [16]:
# procentowe zestawienie ilości kobiet, które otrzymały nagrodę Nobla
# prepared plot
x = nobel_laureates['Year']
y = nobel_laureates['Category']
z = nobel_laureates['percent_of_female']

#female map
fig = go.Figure()
fig.add_trace(go.Heatmap(x = x,
                         y = y,
                         z = z))
fig.update_layout(title = 'Zestawienie procentowej ilości kobiet, które otrzymały nagrodę Nobla w zależności od roku oraz kategorii')
iplot(fig)

## Analiza otrzymywania nagrody Nobla przez kobiety w zestawieniu zależnym od kategorii jak i roku otrzymania
Poszerzenie analizy o uwzględnienie roku pokazuje, że szczególnie w medycynie, wraz z latami zwiększa się ilość laureatek. Warto tu zaznaczyć, że kobiety zdecydowanie później uzyskały prawo do edukacji. Zmiany w dostępie do edukacji wyższej pojawiły się dopiero na przełomie XIX i XX wieku, a nawet wówczas wiele kobiet uznawało tradycyjny model podziału i nie rozwijały się naukowo tylko zajmowały domem i rodziną. Zakorzeniony przez wiele lat model podziału obowiązków społecznych ze względu na płeć spowodował, że kobiety i tak, nawet jak edukację ukończyły, to nie decydowały się na taką ścieżkę kariery. Mimo, że teraz kobiety w nauce nie dziwią, i pojawia się ich coraz więcej (co widać w powyższym zestawieniu szczególnie w kategorii medycyna). Niestety nawet w XXI wieku można spotkać się z podejściem, że kobieta nie nadaje się do nauki lub będzie w zawodach technicznych lub ścisłych "gorsza" (oparte rówenież na https://funip.pl/droga-kobiet-do-edukacji/).
To prowadzi do konieczności wykonania kolejnej analizy. Należało by też zestawić ilość prac naukowo-badawczych, w których brały udział kobiety do ilości kobiet, które nagrodę Nobla dostały. Jeżeli wśród 10 kobiet zajmujących się nauką jest jedna noblistka, a podobne badania prowadzi 100 mężczyzn, ale wśród nich jest już tylko 2 noblistów, to kobiety nie wyglądają w takim zestawieniu źle.
Kolejnym elementem analizy należało by wziąć pod uwagę pewne ewolucyjne predyspozycje do niektórych funkcji poznawczych. Przykładowo ewolucyjna lepsza wyboraźnia przestrzenna u mężczyzn wynikała z faktu, że to mężczyzna odpowiadał z 'polowanie' czy w późniejszym etapie wybudowanie schronienia. Podobnie u kobiet lepiej wykształciła się wielozadaniowość i umiejętności interpretacji niewerbalnej ludzi, bo głównie zajmowały się dziećmi i musiały wiedziec co im dolega zanim one zaczęły mówić. Zatem chociaż by na tej podstawie, można sądzić, że ewolucyjnie większe predyspozycję do chociażby przedmiotów ścisłych czy technicznych mają mężczyźni. O ile ogół społeczeństwa może być w ten sposób oceniany, to nie należy tego odnosić do poszczególnych jednostek.
Podsumowując. Na mniejszą ilość kobiet w nauce, co przekałada się na mniejszą ilość laureatek nagród Nobla, jako najbardziej prestiżowych, przekładać się może wiele czynników. Począwszy od fakt mniejszego zainteresowania kobiet takimi naukami, przez ewolucyjne predyspozycję jak i w konsekwencji możliwości edukacyjne i podejmowania prac naukowych przez kobiety. Jak widać jednak świat się zmienia, a kobiety coraz częściej pojawiają się w różnego rodzaju badaniach naukowych, gdzie ich wkład w rozwój jest niepodważalny.